In studies on machine learning and statistical analysis, the focus is predominantly on the performance of models in terms of accuracy. While accuracy should typically be the primary concern when evaluating a model, sometimes computational performance considerations are imperative when looking at large data sets or models that are widely deployed to serve large populations of client applications.

**Time series** data sets become so large that you simply cannot perform any analysis - or cannot do them correctly - as they are quite demanding in terms of available computational resources. In these cases, many organizations do the following:

- maximized computational resources (expensive and often wasteful, both economic and environmental);
- conduct a project poorly (insufficient hyperparameter adjustment, insufficient data, etc...);
- they do not create a project;

Neither of these options is satisfactory, especially when you are just starting out with a new data set or new analytical technique. It can be frustrating not knowing whether your failures are the result of bad data, a thorny problem, or a lack of resources. Fortunately, we will cover some workarounds to expand your options for very demanding analyzes or huge data sets.

The purpose of the notebook is to guide you with some considerations on how to reduce the computational resources required for training or inference on a specific model. Most of the time, these questions are specific to a particular data set, as well as the resources you have available and your accuracy and speed goals. In this chapter, we will address these concerns, with the hope that they partially cover the problems you encounter and can inspire future brainstorming. These considerations will come to the fore when you have completed your first few rounds of analysis and modeling and should not be a priority when you are dealing with a problem for the first time. However, when the time comes to put something into production or extend a small research project, you should revisit these concerns frequently.